# Train and evaluate the student without distillation to get a baseline

In [ ]:
!pip install transformers datasets accelerate tensorboard evaluate --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      S

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from datasets import load_dataset, DatasetDict
from transformers import (
    ViTFeatureExtractor, ViTForImageClassification,
    TrainingArguments, Trainer, DefaultDataCollator
)
import evaluate
from sklearn.model_selection import train_test_split

In [ ]:
dataset = load_dataset("cifar10")

train_subset_size = 5000
val_subset_size = 1000
test_subset_size = 2000

train_data = dataset["train"].shuffle(seed=42)
train_indices, val_indices = train_test_split(range(len(train_data)), test_size=val_subset_size, random_state=42)

dataset["train"] = train_data.select(range(train_subset_size))
dataset["validation"] = train_data.select(val_indices)
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(test_subset_size))

# Load the feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained("nateraw/vit-base-patch16-224-cifar10")

# Preprocessing function
def process(examples):
    inputs = feature_extractor(examples["img"], return_tensors="np")
    examples["pixel_values"] = inputs["pixel_values"]
    return examples

# Apply the preprocessing to the dataset
processed_datasets = dataset.map(process, batched=True, remove_columns=["img"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/120M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

...

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/343M [00:00<?, ?B/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# lightweight CNN for CIFAR-10
class SmallCNN(nn.Module):
    def __init__(self, num_classes=10, input_size=224):
        super(SmallCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        self._flattened_size = self._get_flattened_size(input_size)

        self.fc1 = nn.Linear(self._flattened_size, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def _get_flattened_size(self, input_size):
        with torch.no_grad():
            x = torch.zeros(1, 3, input_size, input_size)
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = self.pool(F.relu(self.conv3(x)))
        return x.numel()

    def forward(self, pixel_values, labels=None):
        x = self.pool(F.relu(self.conv1(pixel_values)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        logits = self.fc2(x)

        loss = None
        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)

        return {"logits": logits, "loss": loss}

In [ ]:
# Initialize the small CNN model
num_labels = len(processed_datasets["train"].features["label"].names)
small_cnn_model = SmallCNN(num_classes=num_labels)

# Define Training Arguments
baseline_training_args = TrainingArguments(
    output_dir="small-cnn-baseline",
    num_train_epochs=10,  # Fewer epochs for quick testing
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=1e-3,
    logging_dir=None,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",
    push_to_hub=False
)

In [ ]:
# Evaluation metric
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    acc = accuracy.compute(references=labels, predictions=preds)
    return {"accuracy": acc["accuracy"]}

In [ ]:
data_collator = DefaultDataCollator()

# Trainer for baseline training
baseline_trainer = Trainer(
    model=small_cnn_model,
    args=baseline_training_args,
    train_dataset=processed_datasets["train"],
    eval_dataset=processed_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [5]:
# Train the lightweight CNN baseline
baseline_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,1.955400,1.650326,0.397000
2,1.678300,1.405208,0.489000
3,1.529400,1.204969,0.576000
4,1.360900,1.044740,0.659000
5,1.156800,0.793216,0.738000
6,0.944100,0.544634,0.833000
7,0.710600,0.342877,0.918000
8,0.511000,0.198239,0.956000


Epoch,Training Loss,Validation Loss,Accuracy
1,1.955400,1.650326,0.397000
2,1.678300,1.405208,0.489000
3,1.529400,1.204969,0.576000
4,1.360900,1.044740,0.659000
5,1.156800,0.793216,0.738000
6,0.944100,0.544634,0.833000
7,0.710600,0.342877,0.918000
8,0.511000,0.198239,0.956000
9,0.374400,0.138284,0.972000
10,0.310900,0.107329,0.982000


TrainOutput(global_step=1570, training_loss=1.0531955330235183, metrics={'train_runtime': 5813.7719, 'train_samples_per_second': 8.6, 'train_steps_per_second': 0.27, 'total_flos': 0.0, 'train_loss': 1.0531955330235183, 'epoch': 10.0})

In [6]:
# Evaluate on the test set
baseline_test_results = baseline_trainer.evaluate(processed_datasets["test"])
print("Baseline test accuracy: ", baseline_test_results["eval_accuracy"])

Baseline test accuracy:  0.5265
